In [120]:
import pandas as pd
import pycountry
import numpy as np
import json

# Data preparation

In [167]:
df = pd.read_excel('RTA.xlsx')

# get only active and non-accession RTAs
active = df[(df.Status.map(lambda x: x.lower()[0:8]) == "in force") & (df['Accession?'] == "No")][["RTA Name", "Type", "RTA Composition", "Current signatories", "Original signatories"]]

In [168]:
# https://www.iso.org/obp/ui/#search

def get_country(country):
    
    convert = {
        "Bahrain, Kingdom of": "Bahrain",
        "CÙte d'Ivoire": "Côte d'Ivoire",
        "Faeroe Islands": "Faroe Islands",
        "Palestine": "Palestine, State of",
        "Hong Kong, China": "Hong Kong",
        "Chinese Taipei": "Taiwan",
        "Macao, China": "Macao",
        "Iran": "Iran, Islamic Republic of",
        "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
        "Kuwait, the State of": "Kuwait",
        "Saudi Arabia, Kingdom of": "Saudi Arabia",
        "The Gambia": "Gambia",
        "Falkland Islands (Islas Malvinas)": "Falkland Islands (Malvinas)",
        "Wallis and Futuna Islands": "Wallis and Futuna",
        "British Overseas Territory of Saint Helena, Ascension and Tristan da Cunha": "Saint Helena, Ascension and Tristan da Cunha",
        "Aruba, the Netherlands with respect to": "Aruba",
    }
    if country in convert:
        country = convert[country]
    
    x = pycountry.countries.get(name=country) or pycountry.countries.get(official_name=country) or pycountry.countries.get(common_name=country)
    if (x):
        return (x.alpha_2, "")

    return (None, country)

def namelist_to_list(string):
    codelist = []
    notfoundlist = []
    namelist = string.split(';')
    for country in namelist:
        # Hard-coded... Actually includes Aruba as well, but already in the database separately
        if country.strip() == "Netherlands Antilles":
            codelist = codelist + ["CW", "SX", "BQ"]
        elif country.strip() == "UNMIK/Kosovo":
            codelist.append("XK")
        else:
            (code, notfound) = get_country(country.strip())
            if notfound == "":
                # no problem
                codelist.append(code)
            else:
                notfoundlist.append(notfound)
    return codelist, notfoundlist

def get_countries_from_rta(rtaname):
    if rtaname == "European Union":
        rtaname = "EU Treaty"
        
    filtered_rta = coded[coded['RTA Name'] == rtaname]
    if len(filtered_rta) == 0:
        return [], rtaname
    elif len(filtered_rta) > 1:
        raise ValueError('More than one RTA found with name {}'.format(rtaname))
    elif len(filtered_rta) == 1:
        rta_country_list = filtered_rta.iloc[0].code
        return rta_country_list, ""

def get_countries_from_rta_list(rtalist):
    codelist = []
    notfoundlist = []
    for rtaname in rtalist:
        country_list, notfound = get_countries_from_rta(rtaname)
        if (notfound == ""):
            # no problem
            codelist = codelist + country_list
        else:
            notfoundlist.append(notfound)
    return codelist, notfoundlist

In [169]:
# --- resolve countries ---

x = active["Current signatories"].map(namelist_to_list)

new = pd.DataFrame([[a, b] for a, b in x], columns=["code", "notfound"], index=x.index)
coded = pd.concat([active, new], axis=1)

# --- resolve RTAs ---

x = coded.loc[:, 'notfound'].map(get_countries_from_rta_list)
# x = coded[coded['notfound'].apply(len) > 0].loc[:, 'notfound'].map(get_countries_from_rta_list)

new = pd.DataFrame([[a, b] for a, b in x], columns=["newcode", "newnotfound"], index=x.index)
coded = pd.concat([coded, new], axis=1)

coded['code'] = coded['code'] + coded['newcode']

In [170]:
df = coded[['RTA Name', 'Type', 'code']].rename({"RTA Name": "rta", "Type": "type", "code": "countries"}, axis=1)
df = df.reset_index(drop=True)

# Convert into links

In [171]:
# take row in df and convert to array of links

def row_to_links(countrylist, rta_id):
    tmp = countrylist.copy()
    tmp.sort()
    ret = {}
    for i in range(0, len(tmp)):
        for j in range(i + 1, len(tmp)):
            ret[tmp[i] + "-" + tmp[j]] = {"source": tmp[i], "target": tmp[j], "rtas": [rta_id]}
    return ret

# Get country size

In [172]:
# https://data.worldbank.org/indicator/NY.GDP.MKTP.PP.CD
gdp_raw = pd.read_csv('GDP.csv', header=2)
gdp_raw = gdp_raw.drop(['Unnamed: 65'], axis=1)

def get_latest(row):
    maxcol = max([i for i, b in enumerate(row.notna()) if b])
    return row.iloc[maxcol], row.index[maxcol]

def get_alpha2_from_alpha3(alpha3):
    if alpha3 == "XKX":
        return "XK"
    c = pycountry.countries.get(alpha_3=alpha3)
    if c == None:
        return None
    return c.alpha_2

x = gdp_raw.apply(get_latest, axis=1)

new = pd.DataFrame([[a, b] for a, b in x], columns=["gdp", "gdp_year"], index=x.index)
gdp = pd.concat([gdp_raw, new], axis=1)[['Country Name', 'Country Code', 'gdp', 'gdp_year']]
gdp = gdp.rename({"Country Name": "country", "Country Code": "code"}, axis=1)
gdp['code'] = gdp['code'].apply(get_alpha2_from_alpha3)
gdp = gdp.set_index('code')
gdp.index.name = None

In [173]:
world_gdp = gdp[gdp['country'] == "World"].gdp[0]

In [174]:
gdp[gdp['gdp_year'] == "Indicator Code"]

,country,gdp,gdp_year
GI,Gibraltar,NY.GDP.MKTP.CD,Indicator Code
NaN,Not classified,NY.GDP.MKTP.CD,Indicator Code
MF,St. Martin (French part),NY.GDP.MKTP.CD,Indicator Code
KP,"Korea, Dem. People’s Rep.",NY.GDP.MKTP.CD,Indicator Code
VG,British Virgin Islands,NY.GDP.MKTP.CD,Indicator Code


In [175]:
gdp.loc[gdp['gdp_year'] == "Indicator Code", ['gdp', 'gdp_year']] = np.NaN

# Get region

In [176]:
# https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv
region = pd.read_csv('region.csv', keep_default_na=False)
region = region[['name', 'alpha-2', 'alpha-3', 'region', 'sub-region']]

tmp = pd.Series(["Kosovo", "XK", "XKX", "Europe", "Southern Europe"], index=region.columns)
region = region.append(tmp, ignore_index=True)

region = region.set_index('alpha-2')
region.index.name = None

In [177]:
country_data = region.join(gdp[['gdp', 'gdp_year']])

override_gdp = [
    ["AI", 379584360.4, 2019],
#     AQ                                      Antarctica
#     AX                                   Åland Islands
#     BL                                Saint Barthélemy
#     BQ                Bonaire, Sint Eustatius and Saba
#     BV                                   Bouvet Island
#     CC                         Cocos (Keeling) Islands
    ["CK", 379097592.4, 2019],
#     CX                                Christmas Island
#     EH                                  Western Sahara
    ["FK", 206.4e6, 2015],
#     GF                                   French Guiana
    ["GG", 2.742e9, 2005],
    ["GI", 3.22e9, 2018],
#     GP                                      Guadeloupe
#     GS    South Georgia and the South Sandwich Islands
#     HM               Heard Island and McDonald Islands
#     IO                  British Indian Ocean Territory
    ["JE", 5.004e9, 2015],
    ["KP", 16631e6, 2019],
    ["MF", 561.5e6, 2005],
#     MQ                                      Martinique
    ["MS", 67288888.89, 2019],
#     NF                                  Norfolk Island
    ["NU", 10.01e6, 2003],
    ["PM", 261.3e6, 2015],
#     PN                                        Pitcairn
#     RE                                         Réunion
#     SH    Saint Helena, Ascension and Tristan da Cunha
#     SJ                          Svalbard and Jan Mayen
#     TF                     French Southern Territories
#     TK                                         Tokelau
    ["TW", 610.692e9, 2019],
#     UM            United States Minor Outlying Islands
#     VA                                        Holy See
    ["VG", 1.028e9, 2017],
    ["WF", 195e6, 2005],
#     YT                                         Mayotte
]

override = pd.DataFrame(data=[[x[1], x[2]] for x in override_gdp], index=[x[0] for x in override_gdp], columns=['gdp', 'gdp_year'])
country_data.update(override)

country_data.loc['AQ', 'region'] = "Antarctica"
country_data.loc['AQ', 'sub-region'] = "Antarctica"

country_data.loc[country_data['gdp'].isna(), 'gdp'] = 10
country_data.loc[country_data['gdp_year'].isna(), 'gdp_year'] = 0

country_data = country_data.astype({'gdp_year': 'int32'})

In [178]:
country_data.loc['BO', 'name'] = "Bolivia"
country_data.loc['CD', 'name'] = "Democratic Republic of the Congo"
country_data.loc['IR', 'name'] = "Iran"
country_data.loc['KP', 'name'] = "North Korea"
country_data.loc['KR', 'name'] = "South Korea"
country_data.loc['LA', 'name'] = "Laos"
country_data.loc['FM', 'name'] = "Micronesia"
country_data.loc['MD', 'name'] = "Moldova"
country_data.loc['RU', 'name'] = "Russia"
country_data.loc['TW', 'name'] = "Taiwan"
country_data.loc['TZ', 'name'] = "Tanzania"
country_data.loc['GB', 'name'] = "United Kingdom"
country_data.loc['VE', 'name'] = "Venezuela"
country_data.loc['SY', 'name'] = "Syria"
country_data.loc['PS', 'name'] = "Palestine"

# Export

In [179]:
df.to_json('rtas.json', orient="records")

In [180]:
nodes = {}
for index, row in country_data.iterrows():    
    nodes[row.name] = {
        "id": row.name,
        "name": row['name'],
        "alpha3": row['alpha-3'],
        "region": row['region'],
        "subregion": row['sub-region'],
        "gdp": row['gdp'],
        "gdpyear": row['gdp_year']
    }
    
links = {}
for index, row in df[df.type.apply(lambda x: x.find('PSA') == -1)].iterrows():
    cur_rta = row_to_links(row.countries, index)
    for key, value in cur_rta.items():
        if key in links:
            links[key]['rtas'] = links[key]['rtas'] + value['rtas']
        else:
            links[key] = value

final_obj = {"nodes": nodes, "links": links, "total_gdp": world_gdp}

with open('data.json', 'w') as outfile:
    json.dump(final_obj, outfile)